In [1]:
import os
import glob
import numpy as np
import nibabel as nb
from nipype.interfaces import fsl
import nipype.pipeline.engine as pe
from nilearn import image
from joblib import Parallel, delayed


/home/vanes/anaconda2/envs/analysis/lib/python2.7/site-packages/ipykernel/__main__.py:7: DeprecationWarning: Python2 support is deprecated and will be removed in a future release. Consider switching to Python3.
/home/vanes/anaconda2/envs/analysis/lib/python2.7/site-packages/sklearn/utils/extmath.py:23: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._logistic_sigmoid import _log_logistic_sigmoid
/home/vanes/anaconda2/envs/analysis/lib/python2.7/site-packages/sklearn/metrics/cluster/supervised.py:25: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .expected_mutual_info_fast import expected_mutual_information
/home/vanes/anaconda2/envs/analysis/lib/python2.7/site-packages/sklearn/metrics/pairwise.py:31: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .pairwise_fast import _chi2_kernel_fast, _sparse_manhattan
/home/v

In [2]:
home_dir = '/home/shared/2018/visual/cerebellum_prf/'
der_dir = os.path.join(home_dir,'derivatives')
out_dir = os.path.join(home_dir,'derivatives','pp')
res_out_dir = os.path.join(out_dir,'res')
subs = ['02']#,'02','03']#01','02','03']#,'02','03']
sess = {
    '01':['01','02','03'],
    '02':['01','02','03','04'],
    '03':['01','02','03']
}
# sess = ['03']
space = 'MNI152NLin2009cAsym' # 'T1w' 


In [3]:
def savgol_filter(in_file, polyorder=3, deriv=0, window_length = 120):
    """ Applies a savitsky-golay filter to a nifti-file.

    Fits a savitsky-golay filter to a 4D fMRI nifti-file and subtracts the
    fitted data from the original data to effectively remove low-frequency
    signals.

    Parameters
    ----------
    in_file : str
        Absolute path to nifti-file.
    polyorder : int (default: 3)
        Order of polynomials to use in filter.
    deriv : int (default: 0)
        Number of derivatives to use in filter.
    window_length : int (default: 120)
        Window length in seconds.

    Returns
    -------
    out_file : str
        Absolute path to filtered nifti-file.
    """

    import nibabel as nib
    from scipy.signal import savgol_filter
    import numpy as np
    import os

    data = nib.load(in_file)
    dims = data.shape
    affine = data.affine
    header = data.header
    tr = data.header['pixdim'][4]

    # TR must be in seconds
    if tr < 0.01:
        tr = np.round(tr * 1000, decimals=3)
    if tr > 20:
        tr = tr / 1000.0

    window = np.int(window_length / tr)

    # Window must be odd
    if window % 2 == 0:
        window += 1

    data = data.get_data().reshape((np.prod(data.shape[:-1]), data.shape[-1]))
    data_filt = savgol_filter(data, window_length=window, polyorder=polyorder,
                              deriv=deriv, axis=1, mode='nearest')

    data_filt = data - data_filt + data_filt.mean(axis=-1)[:, np.newaxis]
    data_filt = data_filt.reshape(dims)
    img = nib.Nifti1Image(data_filt, affine=affine, header=header)

    return img

In [4]:
def perform_filtering(input_dir,sj_sgtf_out_dir,fn):
    
    # now apply filtering
    out_fn = fn.replace(input_dir,sj_sgtf_out_dir).replace('.nii.gz','_sgtf.nii.gz')

    if not os.path.isfile(out_fn):

        # savitzky golay filtering
        print 'now savgolling %s'%fn
        sg_img = savgol_filter(fn)
        nb.save(sg_img,out_fn)

### perform temporal filtering

In [5]:
# apply savitzky golay filtering

sgtf_out_dir = os.path.join(out_dir,'sgtf')
if not os.path.isdir(sgtf_out_dir): os.mkdir(sgtf_out_dir)

for sub in subs:
    for ses in sess[sub]:
        
        print('now removing drifts in sub %s, ses %s'%(sub,ses))

        # get input fns
        input_dir = os.path.join(res_out_dir,'sub-%s'%sub)
        fns = sorted(glob.glob(os.path.join(input_dir,'sub-%s_ses-%s*bold_space-%s_preproc_resampled_fnirted_smoothed.nii.gz'%(sub,ses,space))))

        # and output
        output_dir = os.path.join(sgtf_out_dir,'sub-%s'%sub)
        if not os.path.isdir(output_dir): os.mkdir(output_dir)

        # now apply filtering in parallel over runs
        Parallel(n_jobs=6,verbose=9)(delayed(perform_filtering)(input_dir,output_dir,fn)  for fn in fns)

print 'done!'

now removing drifts in sub 02, ses 01


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   4 out of  13 | elapsed:  1.3min remaining:  3.0min
[Parallel(n_jobs=6)]: Done   6 out of  13 | elapsed:  1.3min remaining:  1.6min
[Parallel(n_jobs=6)]: Done   8 out of  13 | elapsed:  2.4min remaining:  1.5min
[Parallel(n_jobs=6)]: Done  10 out of  13 | elapsed:  2.4min remaining:   44.0s
[Parallel(n_jobs=6)]: Done  13 out of  13 | elapsed:  3.5min finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.


now removing drifts in sub 02, ses 02


[Parallel(n_jobs=6)]: Done   2 out of   8 | elapsed:  1.2min remaining:  3.7min
[Parallel(n_jobs=6)]: Done   3 out of   8 | elapsed:  1.3min remaining:  2.1min
[Parallel(n_jobs=6)]: Done   4 out of   8 | elapsed:  1.3min remaining:  1.3min
[Parallel(n_jobs=6)]: Done   5 out of   8 | elapsed:  1.3min remaining:   45.3s
[Parallel(n_jobs=6)]: Done   6 out of   8 | elapsed:  1.3min remaining:   25.3s
[Parallel(n_jobs=6)]: Done   8 out of   8 | elapsed:  2.3min remaining:    0.0s
[Parallel(n_jobs=6)]: Done   8 out of   8 | elapsed:  2.3min finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.


now removing drifts in sub 02, ses 03


[Parallel(n_jobs=6)]: Done   3 out of  10 | elapsed:  1.1min remaining:  2.6min
[Parallel(n_jobs=6)]: Done   5 out of  10 | elapsed:  1.1min remaining:  1.1min
[Parallel(n_jobs=6)]: Done   7 out of  10 | elapsed:  2.3min remaining:   59.6s
[Parallel(n_jobs=6)]: Done  10 out of  10 | elapsed:  2.3min finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.


now removing drifts in sub 02, ses 04


[Parallel(n_jobs=6)]: Done   6 out of  15 | elapsed:  1.1min remaining:  1.6min
[Parallel(n_jobs=6)]: Done   8 out of  15 | elapsed:  2.1min remaining:  1.9min
[Parallel(n_jobs=6)]: Done  10 out of  15 | elapsed:  2.1min remaining:  1.1min
[Parallel(n_jobs=6)]: Done  12 out of  15 | elapsed:  2.2min remaining:   32.5s


done!


[Parallel(n_jobs=6)]: Done  15 out of  15 | elapsed:  3.3min finished
